## Fasttext
- word2vec은 사전에 존재하지 않는 단어를 잡아내지 못하는 단점이 있다.
- Fasttext를 통해 이러한 OOV 문제를 해결하며 준수한 성능까지 유도할 수 있을지 알아보자.

## Import

In [1]:
import pandas as pd
import numpy as np

from konlpy.tag import Mecab
from gensim.models import FastText

### Datas

In [2]:
df1 = pd.read_csv("data/maeili.csv", encoding='utf-8')
df2 = pd.read_csv("data/baby.csv", encoding='utf-8')
df3 = pd.read_csv("data/toys.csv", encoding='utf-8')
df4 = pd.read_csv("data/seoulchildcare.csv", encoding='utf-8')
df5 = pd.read_csv("data/babynews_DLC.csv", encoding='utf-8')
df6 = pd.read_csv("data/ildongfoodis.csv", encoding='utf-8')
df7 = pd.read_csv("data/momsmagazine.csv", encoding='utf-8')
df8 = pd.read_csv("data/ange.csv", encoding='utf-8')

### Concatenate Dataframe

In [3]:
df = pd.concat([df1.text, df2.text, df3.text, df4.text, 
                df5.text, df6.text, df7.text, df8.text])
df = pd.DataFrame(df).reset_index(drop=True)
df.tail()

,text
17339,아이는 현재 신체와 정신이 모두 발달 중에 있다 신체 발달은 근육의 발달을 정신 발...
17340,세 살 적 버릇이 여든까지 간다는 말이 있다 밥상머리 예절도 예외는 아니다 아이에게...
17341,아이의 키가 자라고 몸무게가 늘어나면서 아이 몸을 구성하고 있는 장기들도 커지고 무...
17342,눈치는 직관적이고 비언어적인 방법으로 의사소통을 하고 문제를 해결하는 놀라운 능력이...
17343,소리를 들으면 사람은 그 소리를 그대로 흉내 낼 수 있다 이것이 청력의 힘이다 말소...


### Preprocessing

In [4]:
import re

def cleaningText(text):
    # 전처리 하는 순서 중요!!
    text = re.sub('&nbsp;|\xa0|\u200b|\r|\n|\t', ' ', text)
    text = re.sub('<.+?>', '', text)
    text = re.sub('[-_=+,/\?;:^$.@*\"※~&%ㆍ!”“’「」『』〈〉\\‘|\(\)\{\}\[\]\<\>`\'…《》]', '', text)
    text = re.sub('[㈜～→•▪∘○□◇△▽▷◁☆◈◐▶▼▲◆★♦■♥️●♠🖤–]', '', text)
    text = re.sub('http.+com|http.+co|http.+kr|www.+kr|www.+com|www.+net|http.+net|', '', text)
    
    text = re.sub('\s+', ' ', text)
    text = text.strip()

    return text

In [5]:
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])
df = df.drop_duplicates()
df = df.dropna()
df = pd.DataFrame(df).reset_index(drop=True)

In [6]:
df['text'] = df['text'].apply(cleaningText)
df = df.dropna()

### FastText 학습에 적합한 말뭉치로 정제하는 과정
- 한글을 자음, 모음 단위로 분해
- 단어 단위로 구분된 ndarray 자료구조로 정제

In [7]:
import re
from soynlp.hangle import decompose, compose

def remove_doublespace(s):
    doublespace_pattern = re.compile('\s+')
    return doublespace_pattern.sub(' ', s).strip()

def encode(s):
    def process(c):
        if c == ' ':
            return c
        jamo = decompose(c)
        # 'a' or 모음 or 자음
        if (jamo is None) or (jamo[0] == ' ') or (jamo[1] == ' '):
            return ' '
        base = jamo[0]+jamo[1]
        if jamo[2] == ' ':
            return base + '-'
        return base + jamo[2]

    s = ''.join(process(c) for c in s)
    return remove_doublespace(s).strip()

#     s = [process(c) for c in s]
#     return s

def decode(s):
    def process(t):
        assert len(t) % 3 == 0
        t_ = t.replace('-', ' ')
        chars = [tuple(t_[3*i:3*(i+1)]) for i in range(len(t_)//3)]
        recovered = [compose(*char) for char in chars]
        recovered = ''.join(recovered)
        return recovered

    return ' '.join(process(t) for t in s.split())

#### Test

In [8]:
s = '나의 야근을 알까? 흐헤헤 흐헤헤 아이고난1 아이고난2'
print(encode(s))
print(decode(encode(s)))

ㄴㅏ-ㅇㅢ- ㅇㅑ-ㄱㅡㄴㅇㅡㄹ ㅇㅏㄹㄲㅏ- ㅎㅡ-ㅎㅔ-ㅎㅔ- ㅎㅡ-ㅎㅔ-ㅎㅔ- ㅇㅏ-ㅇㅣ-ㄱㅗ-ㄴㅏㄴ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㄴㅏㄴ
나의 야근을 알까 흐헤헤 흐헤헤 아이고난 아이고난


In [9]:
df['encoded'] = df['text'].apply(encode)
df

,text,encoded
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...,ㅅㅏㄹ ㄴㅏㅁㅇㅏ- ㅇㅣㅂㄴㅣ-ㄷㅏ- ㅎㅚ-ㅅㅏ- ㅂㅗㄱㅈㅣㄱ ㅎㅜ- ㅅㅏㄹㅂㅜ-ㅌ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...,ㅇㅛ-ㅈㅡㅁ ㄷㅡㄹㅇㅓ- ㅇㅏ-ㅃㅏ-ㄱㅏ- ㅇㅏ-ㅇㅣ-ㅇㅔ-ㄱㅔ- ㄷㅏ-ㄱㅏ-ㄱㅏ-...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...,ㄷㅜㄹㅉㅐ-ㄱㅏ- ㅌㅐ-ㅇㅓ-ㄴㅏ-ㅁㅕㄴ ㅊㅓㅅㅉㅐ-ㄱㅏ- ㅎㅣㅁㄷㅡㄹ ㄱㅓ-ㄹㅏ-...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...,ㄱㅐ-ㅇㅝㄹ ㄴㅏㅁㅇㅏ-ㄱㅏ- ㅇㅣㅆㄱㅗ- ㅇㅣㅁㅅㅣㄴ ㄱㅐ-ㅇㅝㄹㅊㅏ-ㄹㅗ- ㄷㅜ...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...,ㄱㅐ-ㅇㅝㄹ ㄴㅏㅁㅈㅏ- ㅇㅏ-ㄱㅣ-ㅇㅣㄴㄷㅔ- ㅅㅣㅁㅎㅏ-ㄱㅔ- ㄲㅐ-ㅁㅜ-ㄴㅡㄴ...
...,...,...
16261,홍삼은 자양강장 피로 해소 식욕 증진 등 다양한 효능으로 한국인이 선호하는 대표 건...,ㅎㅗㅇㅅㅏㅁㅇㅡㄴ ㅈㅏ-ㅇㅑㅇㄱㅏㅇㅈㅏㅇ ㅍㅣ-ㄹㅗ- ㅎㅐ-ㅅㅗ- ㅅㅣㄱㅇㅛㄱ ㅈㅡ...
16262,기저귀 발진은 기저귀 안쪽과 기저귀가 닿는 부위인 엉덩이 성기 사타구니등이 붉게 변...,ㄱㅣ-ㅈㅓ-ㄱㅟ- ㅂㅏㄹㅈㅣㄴㅇㅡㄴ ㄱㅣ-ㅈㅓ-ㄱㅟ- ㅇㅏㄴㅉㅗㄱㄱㅘ- ㄱㅣ-ㅈㅓ-...
16263,아구창 생후 6개월 미만의 아기에게 많이 생기는 병이다 아기가 태어날 때엄마의 질에...,ㅇㅏ-ㄱㅜ-ㅊㅏㅇ ㅅㅐㅇㅎㅜ- ㄱㅐ-ㅇㅝㄹ ㅁㅣ-ㅁㅏㄴㅇㅢ- ㅇㅏ-ㄱㅣ-ㅇㅔ-ㄱㅔ-...
16264,흔히 ADHD라 불리는 주의력결핍과잉행동장애는 아이 스스로 충동을 참지 못하고 집중...,ㅎㅡㄴㅎㅣ- ㄹㅏ- ㅂㅜㄹㄹㅣ-ㄴㅡㄴ ㅈㅜ-ㅇㅢ-ㄹㅕㄱㄱㅕㄹㅍㅣㅂㄱㅘ-ㅇㅣㅇㅎㅐㅇㄷ...


In [10]:
encoded_corpus = np.array(df.encoded)
type(encoded_corpus), len(encoded_corpus)

(numpy.ndarray, 16266)

### Train

In [11]:
%%time

class FastTextCorpus:
    def __init__(self, corpus):
        self.corpus = corpus
        self.n_iter = 0
    def __iter__(self):
        for i, doc in enumerate(self.corpus):
            if i % 500 == 0 and i > 0:
                print('\riter = {}, sents = {} ...'.format(self.n_iter, i), end='')
            yield doc.split()
        self.n_iter += 1
        print('\riter = {}, docs = {} done{}'.format(self.n_iter, i, ' '*20))
        
corpus = FastTextCorpus(encoded_corpus)
fasttext_model = FastText(
    corpus,
    size=100,
    seed=191212,
    workers=6,
    window=3,
    min_count=6,
    min_n=3,
    max_n=6,
)

iter = 1, docs = 16265 done                    
iter = 2, docs = 16265 done                    
iter = 3, docs = 16265 done                    
iter = 4, docs = 16265 done                    
iter = 5, docs = 16265 done                    
iter = 6, docs = 16265 done                    
CPU times: user 3min 53s, sys: 898 ms, total: 3min 54s
Wall time: 1min


In [12]:
fasttext_model.save('data/tokmom_fasttext.model')

In [13]:
def most_similar(query, topn=10):
    query_ = encode(query)
    similars = fasttext_model.wv.most_similar(query_, topn=topn)
    similars = [(decode(word), sim) for word, sim in similars]
    return similars

In [14]:
most_similar("여자아이")

[('남자아이', 0.9517726898193359),
 ('첫아이', 0.9146806001663208),
 ('셋째아이', 0.9039974808692932),
 ('경희아이', 0.8905543088912964),
 ('큰아이', 0.8739292621612549),
 ('딸아이', 0.8622821569442749),
 ('우리아이', 0.8570424914360046),
 ('신비아이', 0.8546150922775269),
 ('빠이빠이', 0.8535259366035461),
 ('여자아이가', 0.852910041809082)]

In [15]:
most_similar("여아")

[('역아', 0.8313663005828857),
 ('영아', 0.8204909563064575),
 ('최영아', 0.796942949295044),
 ('홍승아', 0.7889565229415894),
 ('퀴노아', 0.7864316701889038),
 ('쌍생아', 0.7822641134262085),
 ('쏘아', 0.7802329063415527),
 ('형아', 0.7775528430938721),
 ('뮤아', 0.7742391228675842),
 ('아쿠아', 0.7639589309692383)]

In [16]:
most_similar("남자아이")

[('여자아이', 0.9517726898193359),
 ('첫아이', 0.9229631423950195),
 ('셋째아이', 0.9209100604057312),
 ('경희아이', 0.8901501893997192),
 ('큰아이', 0.8852757215499878),
 ('나이', 0.8649623394012451),
 ('딸아이', 0.8637534379959106),
 ('남자아이가', 0.8561583757400513),
 ('아이', 0.8510080575942993),
 ('빠이빠이', 0.8500628471374512)]

In [17]:
most_similar("남아")

[('삼아', 0.82341468334198),
 ('살아남아', 0.8221031427383423),
 ('남겨져', 0.782551646232605),
 ('남겨', 0.7770277261734009),
 ('동남아', 0.7656135559082031),
 ('쌓아', 0.7642712593078613),
 ('녹아', 0.7355770468711853),
 ('닿아', 0.7342108488082886),
 ('닳아', 0.7322394251823425),
 ('닮아', 0.7238664627075195)]

In [18]:
most_similar("치아")

[('소피아', 0.8282185792922974),
 ('치아나', 0.8060892224311829),
 ('무뇌아', 0.804297924041748),
 ('퀴노아', 0.7806976437568665),
 ('쏘아', 0.7801614999771118),
 ('미아', 0.7738290429115295),
 ('치아관리', 0.7560563087463379),
 ('소아', 0.7535560727119446),
 ('치아건강', 0.7405077219009399),
 ('콜롬비아', 0.7402966022491455)]

In [19]:
most_similar("이빨")

[('이탈', 0.9203110933303833),
 ('이어갈', 0.872407078742981),
 ('이혼할', 0.8710523843765259),
 ('이길', 0.8518725037574768),
 ('이룰', 0.8287208080291748),
 ('이겨낼', 0.809754490852356),
 ('이질', 0.8039109706878662),
 ('이럴', 0.801956295967102),
 ('이행할', 0.7988680601119995),
 ('이날', 0.7970359325408936)]

In [20]:
most_similar("놀이")

[('책놀이', 0.9665970802307129),
 ('윷놀이', 0.9605213403701782),
 ('공놀이', 0.9587685465812683),
 ('까꿍놀이', 0.9532339572906494),
 ('소꿉놀이', 0.953145444393158),
 ('탈놀이', 0.9510608911514282),
 ('영어놀이', 0.9398317933082581),
 ('점토놀이', 0.939724862575531),
 ('투호놀이', 0.9381253719329834),
 ('모래놀이', 0.9373698830604553)]

In [21]:
most_similar("감기")

[('열감기', 0.9363807439804077),
 ('코감기', 0.9250694513320923),
 ('염기', 0.896769106388092),
 ('여름감기', 0.8591163158416748),
 ('환절기', 0.8535810708999634),
 ('심기', 0.8437669277191162),
 ('비뇨기', 0.838300347328186),
 ('남기', 0.8382030129432678),
 ('감기약', 0.8375320434570312),
 ('감염되기', 0.8373985290527344)]

### 결과 확인
하이퍼 파라미터 조정을 다양하게 시도해보며 결과를 확인해봤지만, 역시 사용하기 어렵겠다는 결론을 내렸다. 말뭉치가 부족한 것이 원인이 아닐까?

---

## 못내 아쉬워서 재도전
- 이번엔 자음, 모음 단위가 아닌 글자 단위로 해보자
- 간단하게 Mecab으로 tokenizing한 말뭉치를 학습으로 사용

In [22]:
from konlpy.tag import Mecab

In [23]:
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
    text = ' '.join(text)
    return text

In [24]:
%%time
df['tokenized'] = df['text'].apply(tokenize)
df.tail()

CPU times: user 38.8 s, sys: 19 s, total: 57.8 s
Wall time: 56.6 s


,text,encoded,tokenized
16261,홍삼은 자양강장 피로 해소 식욕 증진 등 다양한 효능으로 한국인이 선호하는 대표 건...,ㅎㅗㅇㅅㅏㅁㅇㅡㄴ ㅈㅏ-ㅇㅑㅇㄱㅏㅇㅈㅏㅇ ㅍㅣ-ㄹㅗ- ㅎㅐ-ㅅㅗ- ㅅㅣㄱㅇㅛㄱ ㅈㅡ...,홍삼 은 자양 강장 피로 해소 식욕 증진 등 다양 한 효능 으로 한국인 이 선호 하...
16262,기저귀 발진은 기저귀 안쪽과 기저귀가 닿는 부위인 엉덩이 성기 사타구니등이 붉게 변...,ㄱㅣ-ㅈㅓ-ㄱㅟ- ㅂㅏㄹㅈㅣㄴㅇㅡㄴ ㄱㅣ-ㅈㅓ-ㄱㅟ- ㅇㅏㄴㅉㅗㄱㄱㅘ- ㄱㅣ-ㅈㅓ-...,기저귀 발진 은 기저귀 안쪽 과 기저귀 가 닿 는 부위 인 엉덩이 성기 사타구니 등...
16263,아구창 생후 6개월 미만의 아기에게 많이 생기는 병이다 아기가 태어날 때엄마의 질에...,ㅇㅏ-ㄱㅜ-ㅊㅏㅇ ㅅㅐㅇㅎㅜ- ㄱㅐ-ㅇㅝㄹ ㅁㅣ-ㅁㅏㄴㅇㅢ- ㅇㅏ-ㄱㅣ-ㅇㅔ-ㄱㅔ-...,아구창 생후 6 개월 미만 의 아기 에게 많이 생기 는 병 이 다 아기 가 태어날 ...
16264,흔히 ADHD라 불리는 주의력결핍과잉행동장애는 아이 스스로 충동을 참지 못하고 집중...,ㅎㅡㄴㅎㅣ- ㄹㅏ- ㅂㅜㄹㄹㅣ-ㄴㅡㄴ ㅈㅜ-ㅇㅢ-ㄹㅕㄱㄱㅕㄹㅍㅣㅂㄱㅘ-ㅇㅣㅇㅎㅐㅇㄷ...,흔히 ADHD 라 불리 는 주의력 결핍 과잉 행동 장애 는 아이 스스로 충동 을 참...
16265,아이의 키가 자라고 몸무게가 늘어나면서 아이 몸을 구성하고 있는 장기들도 커지고 무...,ㅇㅏ-ㅇㅣ-ㅇㅢ- ㅋㅣ-ㄱㅏ- ㅈㅏ-ㄹㅏ-ㄱㅗ- ㅁㅗㅁㅁㅜ-ㄱㅔ-ㄱㅏ- ㄴㅡㄹㅇㅓ-...,아이 의 키 가 자라 고 몸무게 가 늘어나 면서 아이 몸 을 구성 하 고 있 는 장...


In [25]:
encoded_corpus = np.array(df.tokenized)
len(encoded_corpus)

16266

In [26]:
%%time
class FastTextCorpus:
    def __init__(self, corpus):
        self.corpus = corpus
        self.n_iter = 0
    def __iter__(self):
        for i, doc in enumerate(self.corpus):
            if i % 500 == 0 and i > 0:
                print('\riter = {}, sents = {} ...'.format(self.n_iter, i), end='')
            yield doc.split()
        self.n_iter += 1
        print('\riter = {}, docs = {} done{}'.format(self.n_iter, i, ' '*20))
        
corpus = FastTextCorpus(encoded_corpus)
fasttext_token_model = FastText(
    corpus,
    size=100,
    seed=191212,
    workers=6,
    window=5,
    min_count=8,
    min_n=3,
    max_n=6,
)

iter = 1, docs = 16265 done                    
iter = 2, docs = 16265 done                    
iter = 3, docs = 16265 done                    
iter = 4, docs = 16265 done                    
iter = 5, docs = 16265 done                    
iter = 6, docs = 16265 done                    
CPU times: user 1min 52s, sys: 943 ms, total: 1min 53s
Wall time: 32.9 s


In [27]:
fasttext_token_model.save('data/tokmom_token_fasttext.model')

In [28]:
fasttext_token_model.wv.most_similar("여아")

[('남아', 0.894153356552124),
 ('남아나', 0.7436610460281372),
 ('남자', 0.6159439086914062),
 ('경산부', 0.6079232096672058),
 ('여자', 0.5924214124679565),
 ('초산부', 0.5894688367843628),
 ('조산아', 0.5759049654006958),
 ('IQ', 0.546113133430481),
 ('47', 0.5432536602020264),
 ('성인기', 0.5431994199752808)]

In [29]:
fasttext_token_model.wv.most_similar("여자아이")

[('친구', 0.45197927951812744),
 ('엄마', 0.4485394060611725),
 ('어린아이', 0.44812124967575073),
 ('부모', 0.4480510354042053),
 ('아이', 0.4465680718421936),
 ('남자', 0.44166475534439087),
 ('어른', 0.4371682107448578),
 ('첫아이', 0.4371494948863983),
 ('여자', 0.4234277009963989),
 ('외동아이', 0.4231249690055847)]

In [30]:
fasttext_token_model.wv.most_similar("남아")

[('여아', 0.894153356552124),
 ('남아나', 0.7657767534255981),
 ('경산부', 0.6163709163665771),
 ('IQ', 0.5989770889282227),
 ('여자', 0.5985745191574097),
 ('초산부', 0.5983076095581055),
 ('남자', 0.5932149887084961),
 ('조산아', 0.5890449285507202),
 ('성인기', 0.5803169012069702),
 ('몸무게', 0.5701994895935059)]

In [31]:
fasttext_token_model.wv.most_similar("남자아이")

[('소유', 0.5462145805358887),
 ('사나이', 0.5190684795379639),
 ('나이', 0.5190239548683167),
 ('남자', 0.4952504634857178),
 ('인종', 0.4844752550125122),
 ('거짓말', 0.48178350925445557),
 ('소유욕', 0.4796436131000519),
 ('뼈나이', 0.4727550745010376),
 ('성인식', 0.4635210633277893),
 ('여자', 0.46203160285949707)]

In [32]:
fasttext_token_model.wv.most_similar("치아")

[('잇몸', 0.8022342920303345),
 ('영구치', 0.7788367867469788),
 ('치아우식증', 0.7164109945297241),
 ('유치', 0.6943153738975525),
 ('뼈', 0.6900635957717896),
 ('모발', 0.68394935131073),
 ('충치', 0.6478229761123657),
 ('어금니', 0.6332655549049377),
 ('혀', 0.6282088756561279),
 ('치석', 0.6233335733413696)]

In [33]:
fasttext_token_model.wv.most_similar("이빨")

[('혓바늘', 0.7611755132675171),
 ('얼음물', 0.7311898469924927),
 ('절구', 0.722432017326355),
 ('속껍질', 0.7130637764930725),
 ('꽁꽁', 0.7108073234558105),
 ('뒷목', 0.7086732387542725),
 ('얼음찜질', 0.707527220249176),
 ('행주', 0.6995844841003418),
 ('돗바늘', 0.6970102787017822),
 ('겉껍질', 0.6968401670455933)]

In [34]:
fasttext_token_model.wv.most_similar("놀이")

[('길놀이', 0.8846937417984009),
 ('공놀이', 0.8697115182876587),
 ('투호놀이', 0.8281128406524658),
 ('탈놀이', 0.8201621770858765),
 ('기차놀이', 0.8183891177177429),
 ('불꽃놀이', 0.7919838428497314),
 ('놀이마당', 0.766674280166626),
 ('풍물놀이', 0.7616157531738281),
 ('윷놀이', 0.760199785232544),
 ('놀이기구', 0.7575340867042542)]

In [35]:
fasttext_token_model.wv.most_similar("이빠ㄹ")

[('생으로', 0.3613571226596832),
 ('으로', 0.32532399892807007),
 ('여지', 0.32144108414649963),
 ('참으로', 0.3156202435493469),
 ('실마리', 0.31076550483703613),
 ('예상', 0.30294835567474365),
 ('0', 0.3000352382659912),
 ('그대', 0.29037222266197205),
 ('패스트', 0.2814841568470001),
 ('선', 0.2785693407058716)]

In [36]:
fasttext_token_model.wv.most_similar("놀ㅇ")

[('차질', 0.33836987614631653),
 ('돼', 0.3072327971458435),
 ('도표', 0.29358774423599243),
 ('가능', 0.2904548943042755),
 ('무독', 0.28529757261276245),
 ('급속히', 0.28417372703552246),
 ('예외', 0.28130602836608887),
 ('급속도', 0.27952927350997925),
 ('지연', 0.27931344509124756),
 ('기구', 0.27215099334716797)]

In [37]:
fasttext_token_model.wv.most_similar("아기")

[('아기집', 0.8885033130645752),
 ('갓난아기', 0.8742845058441162),
 ('아이', 0.8212771415710449),
 ('아기방', 0.8006928563117981),
 ('신생아기', 0.7975958585739136),
 ('태아기', 0.7861014604568481),
 ('갓난아이', 0.7549307346343994),
 ('신생아', 0.7478479743003845),
 ('첫아이', 0.7441725730895996),
 ('아기과자', 0.7059472799301147)]

In [38]:
fasttext_token_model.wv.most_similar("애기")

[('엉엉', 0.7407138347625732),
 ('멋쟁이', 0.7151405811309814),
 ('야지', 0.706068754196167),
 ('부엉이', 0.6928058862686157),
 ('멍멍이', 0.6890251636505127),
 ('귀여워서', 0.6885696649551392),
 ('동갑내기', 0.6883336305618286),
 ('이따', 0.6877667903900146),
 ('했잖아', 0.6867496967315674),
 ('래요', 0.6863689422607422)]

In [39]:
fasttext_token_model.wv.most_similar("아ㄱ")

[('옥시토신', 0.37168291211128235),
 ('고심', 0.3561030626296997),
 ('환원', 0.3554624021053314),
 ('세계인', 0.3554267883300781),
 ('훌륭', 0.35406509041786194),
 ('방목', 0.3536190390586853),
 ('세계관', 0.3523695468902588),
 ('활발히', 0.35197538137435913),
 ('기이', 0.3439064621925354),
 ('방대', 0.34291988611221313)]

In [40]:
fasttext_token_model.wv.most_similar("찡")

[('찡그린', 0.7638769745826721),
 ('깜박임', 0.7611411809921265),
 ('삐뚤빼뚤', 0.7576819658279419),
 ('어설픈', 0.7471916675567627),
 ('끄덕임', 0.7406378984451294),
 ('두근두근', 0.7346431016921997),
 ('내민', 0.7345788478851318),
 ('커다랗', 0.7272733449935913),
 ('꺄르르', 0.7239421606063843),
 ('풀빛', 0.7178086042404175)]

In [41]:
fasttext_token_model.wv.most_similar("열")

[('미열', 0.6306690573692322),
 ('배탈', 0.5792239308357239),
 ('고열', 0.5637903809547424),
 ('땀', 0.5555974841117859),
 ('진물', 0.5442134141921997),
 ('콧물', 0.540371835231781),
 ('식은땀', 0.5300861597061157),
 ('뇌간', 0.525671660900116),
 ('코피', 0.5254379510879517),
 ('큰일', 0.5185173749923706)]

### Review


- 이전 모델이 비해 전체적인 성능이 더 좋아진 것은 체감된다!
- 하지만 오타나 말뭉치에 존재하지 않는 단어를 입력받을 경우, 최소 error는 발생하진 않지만 성능에 대한 이슈는 여전히 존재한다 
  - 보통 오타는 자음 모음을 누락하거나 잘못 입력하는 경우에 발생하기 때문. 글자 단위로 임베딩 했을 때는 이런 부분을 잡기 어렵다.
  - 어떻게든 결과는 나오지만 fasttext만의 특색이자 장점을 못살리는 느낌.. 
- 그리고 fasttext 특성 상 "남아"의 유사어로 "동남아"가 나오는 식의 문제는 어떻게 할 수가 없다; 이것은 자음 모음 단위로 임베딩 할 경우에도 있었던 문제임
- Embedding 결과 성능을 정량적으로 비교해서 word2vec과 fasttext 둘 중 무엇이 좋을지 확실히 비교할 수 있는 방안을 찾아서 어떤 것을 활용할지 결정해야 할 듯